# Mapping AACN Sub-Competencies Using Semantic Similarity Scores

## Introduction

* I'm Chris Macintosh
* Director of Nursing Informatics MS at University of Utah College of Nursing
* I'm going to demonstrate how to generate semantic similarity scores (cosine similarity) using an open-source large language model (LLM)

## Semantic Similarity

* The lines between quantitative and qualitiative analysis are becomming blurred.
* LLMs can be used to perform quantitative analysis on similarity in meaning of textual data.
* This is a demo of calculating semantic similarity scores for different groups of sentences.

## AACN Essentials

* AACN outlined 10 domains for undergraduate and graduate level nursing competencies.
* Those 10 domains contain 45 competencies.
* The 45 competencies are further defined into sub-competencies.
* There are 204 graduate-level sub-competencies.
* Mapping that many sub-competencies is not a trivial task.
* This proof of concept report maps AACN subcompetencies to course learning outcomes by ranking them by cosine similarity score.

## Sentence Embedding

* Sentence embedding involves replacing words with arrays of numbers called vectors.
* Mathematical techniques can be done to compare vectors with each other.
* Older methods used sparce vectors that consisted mostly of zeros.
* Older methods called "bag of words" focused on words present, but ignored word order.
* More sophisticated methods like if-idf (term frequency - inverse document frequency) and BM25 (best match 25) expanded on "bag of words" and incorporated significance of words.
* More sophisticated neural network methods result in dense vectors (non-zero vectors) that identify location of a sentence in a large vector space.
* Dense vector methods can encode semantics.
* BERT (bidirectional encoder representations from transformers) also encode how words relate to each other.
* SBERT (sentence - BERT) is an improvement on BERT that speeds up comparisons.

## Overview of the Steps

* Sentence vector embeddings are created.
* Cosine similarity scores are calculated.
* Scores are exported to and Excel workbook so formatting could be used to highlight important aspects and lists could be sorted.

## Google Colaboratory

* Code is run on a Jupyter Notebook in Google Colab.

## File Preparation

* I created a data file using Excel.
* A separate sheet for each group of sentences was created.
* Each sentence was assigned a unique identifier.
* Both sheets have an id column and a description column.
* This could be done using any two groups of sentences.

## Options for Loading Files

Select one of the options below.

### Option #1: Clone GitHub Repository



In [ ]:
!git clone https://github.com/cmcntsh/SemanticSimilarityReport_ALNI_NIWG_2024-09.git # replace the address shown with the address to your own repository

### Option #2: Use Command in Jupyter Notebook

In [ ]:
from google.colab import files
uploaded = files.upload()

### Option #3: Use Google Colab Interface

(Not demonstrated here.)

### Option #4: Upload from Google Drive

(Not demonstrated here.)

## File Path

In [ ]:
file_path = '/content/SemanticSimilarityReport_ALNI_NIWG_2024-09/InformaticsCompetenciesSheet.xlsx'

## Install Sentence Transformers

In [ ]:
# Only needs to run if not already installed.
!pip install sentence-transformers

## Read File

In [ ]:
# Import data from Excel workbook.
# There should be two sheets (Sheet1, Sheet2) with headings id & description.

import pandas as pd

DataFile = file_path

df1 = pd.read_excel(DataFile, 'Sheet1')
df2 = pd.read_excel(DataFile, 'Sheet2')

# Add the ids and text chunks to lists.
Sheet1_id = df1.id.values.tolist()
Sheet1_text = df1.description.values.tolist()
Sheet2_id = df2.id.values.tolist()
Sheet2_text = df2.description.values.tolist()

## Import the Model

In [ ]:
from sentence_transformers import SentenceTransformer

# Models - https://huggingface.co/models?library=sentence-transformers
model = SentenceTransformer('all-mpnet-base-v2')

## Create Sentence Embeddings

In [ ]:
embeddings1 = model.encode(Sheet1_text)
embeddings2 = model.encode(Sheet2_text)

## Calculate Cosine Similarity Scores

In [ ]:
from sentence_transformers.util import cos_sim

scores = cos_sim(embeddings1, embeddings2)

## Create Dataframes

In [ ]:
import numpy as np

scores_df = pd.DataFrame(scores.numpy(), index = Sheet1_id, columns = Sheet2_id)
scores_df
transposed_df = scores_df.transpose()
#transposed_df

In [ ]:
print(len(scores_df))
print(len(transposed_df))
print(scores_df.shape)

## Generate the Excel Report

In [ ]:
!pip install xlsxwriter

In [ ]:
import os

print(os.path.split(DataFile)[0])

In [ ]:
# Set the path and name for the Excel workbook to create.
OutputFile = os.path.split(DataFile)[0] + "\CosineSimilarity_" + os.path.split(DataFile)[1]


# Determine column letters from column numbers
# https://stackoverflow.com/questions/29351492/how-to-make-a-continuous-alphabetic-list-python-from-a-z-then-from-aa-ab-ac-e
def char_label(n, chars):
    indexes = []
    while n:
        residual = n % len(chars)
        if residual == 0:
            residual = len(chars)
        indexes.append(residual)
        n = (n - residual)
        n = n // len(chars)
    indexes.reverse()
    label = ''
    for i in indexes:
        label += chars[i-1]
    return label

my_chrs = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
#start_col = 2
#end_col = start_col + 204
#start_row = 10
#end_row = start_row + 27
#for col in range(start_col, end_col+1):
#    r=char_label(col, my_chrs)
#    print(r + str(1) + " = max(" + r + str(start_row) + ":" + r + str(end_row) + ")")

# Use a with statement to close the file automatically when all file writing and formatting is complete.
with pd.ExcelWriter(OutputFile) as writer:

    # Write the scores dataframe to the Excel workbook. Leave blank rows at the top.
    scores_df.to_excel(writer, sheet_name="Sheet1", startrow=10, startcol=0)

    # Get the number of rows and columns
    (max_row, max_col) = scores_df.shape

    # Load the sheet as an object to do some formatting later
    sheet1 = writer.sheets['Sheet1']

    # Label the first row as "Max". I'll identify the highest match in Excel later.
    sheet1.write(0,0,"Max")

    # Add formula for Max value in column in first row.
    for col in range(2, max_col+2):
        r=char_label(col, my_chrs)
        sheet1.write(0, col-1, "= max(" + r + str(12) + ":" + r + str(11+max_row) + ")")

    sheet1.conditional_format(0,1, max_row+10, max_col, {"type": "3_color_scale", "mid_color": "#FFFFFF", "max_color": "#008000", "min_color": "#FFFF00", "min_value": -1, "max_value": 1, "mid_value": 0})

    # Set up conditional formatting in the worksheet.
    sheet1.conditional_format(10,1, max_row+10, max_col, {"type": "3_color_scale", "mid_color": "#FFFFFF", "max_color": "#0000FF", "min_color": "#FFFF00", "min_value": -1, "max_value": 1, "mid_value": 0})

    # Add the description text as comments for the rows.
    for i in range(len(Sheet1_text)):
        sheet1.write_comment(i+11,0,Sheet1_text[i], {'text_wrap': 0, "x_scale": 2, "y_scale": 2})

    # Add the description text as comments for the columns.
    for i in range(len(Sheet2_text)):
        sheet1.write_comment(10,i+1,Sheet2_text[i], {'text_wrap': 0, "x_scale": 2, "y_scale": 2})

    # Add the transposed data to a second sheet with same formatting as first sheet.
    transposed_df.to_excel(writer, sheet_name="Sheet2", startrow=10, startcol=0)
    (max_row, max_col) = transposed_df.shape
    sheet2 = writer.sheets['Sheet2']
    sheet2.write(0,0,"Max")

    # Add formula for Max value in column in first row.
    for col in range(2, max_col+2):
        r=char_label(col, my_chrs)
        sheet2.write(0, col-1, "= max(" + r + str(12) + ":" + r + str(11+max_row) + ")")

    sheet2.conditional_format(0,1, max_row+10, max_col, {"type": "3_color_scale", "mid_color": "#FFFFFF", "max_color": "#008000", "min_color": "#FFFF00", "min_value": -1, "max_value": 1, "mid_value": 0})

    sheet2.conditional_format(10,1, max_row+10, max_col, {"type": "3_color_scale", "mid_color": "#FFFFFF", "max_color": "#0000FF", "min_color": "#FFFF00", "min_value": -1, "max_value": 1, "mid_value": 0})
    for i in range(len(Sheet2_text)):
        sheet2.write_comment(i+11,0,Sheet2_text[i], {'text_wrap': 0, "x_scale": 2, "y_scale": 2})
    for i in range(len(Sheet1_text)):
        sheet2.write_comment(10,i+1,Sheet1_text[i], {'text_wrap': 0, "x_scale": 2, "y_scale": 2})
print("Done")

## Download Report